Dataset used:
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. https://doi.org/10.1145/2827872


Implementation based on paper:
Chen, Y. (2025). Contextual bandits to increase user prediction accuracy in movie recommendation system. ITM Web of Conferences, 73, 01018. https://doi.org/10.1051/itmconf/20257301018

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## STEP 1: PREPROCESSING DATA & EXTRACTING USER-MOVIE-FEATURES

In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/AI Planning/u.data", sep="\t", header=None)
df.columns=['user_id', 'item_id', 'rating', 'timestamp']
df.drop('timestamp', axis=1, inplace=True)
df.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [3]:
user = pd.read_csv("/content/drive/MyDrive/AI Planning/u.user", sep="|", header=None)
user.columns=['user_id', 'age', 'gender', 'occupation','zip code']
user.drop('zip code', axis=1, inplace=True)
user.head()

,user_id,age,gender,occupation
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


In [4]:
combined_df = df.merge(user, how='left')
combined_df.head()

,user_id,item_id,rating,age,gender,occupation
0,196,242,3,49,M,writer
1,186,302,3,39,F,executive
2,22,377,1,25,M,writer
3,244,51,2,28,M,technician
4,166,346,1,47,M,educator


In [5]:
movie = pd.read_csv("/content/drive/MyDrive/AI Planning/u.item", sep='|', header=None, encoding='latin-1')


movie.columns=['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
               'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
               'Sci-Fi', 'Thriller', 'War', 'Western']

movie.drop('release date', axis=1, inplace=True)
movie.drop('video release date', axis=1, inplace=True)
movie.drop('IMDb URL', axis=1, inplace=True)
movie.drop('unknown', axis=1, inplace=True)
movie.drop('movie title', axis=1, inplace=True)

movie.head()

,item_id,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [6]:
combined_df = combined_df.merge(movie, how='left')
combined_df.head()

,user_id,item_id,rating,age,gender,occupation,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,49,M,writer,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,39,F,executive,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
2,22,377,1,25,M,writer,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,28,M,technician,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,166,346,1,47,M,educator,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(combined_df[['gender', 'occupation']])

encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['gender', 'occupation']))

for col in encoded.columns:
  encoded[col]=encoded[col].astype(int)

encoded.head()

,gender_F,gender_M,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,occupation_healthcare,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
user_feature = pd.concat([combined_df['age'], encoded], ignore_index=False, sort=False, axis=1)

from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
user_feature['age'] = minmax.fit_transform(user_feature[['age']])

user_feature.head()

,age,gender_F,gender_M,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,...,occupation_marketing,occupation_none,occupation_other,occupation_programmer,occupation_retired,occupation_salesman,occupation_scientist,occupation_student,occupation_technician,occupation_writer
0,0.636364,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.484848,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.272727,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.318182,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.606061,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
user_movie_feature = pd.concat([user_feature, combined_df.iloc[:,6:]], ignore_index=False, sort=False, axis=1)
user_movie_feature.head()

,age,gender_F,gender_M,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.636364,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.484848,1,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,0.272727,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.318182,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
4,0.606061,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# STEP 2: CLUSTERING

In [10]:
print(len(combined_df['user_id'].unique()))
print(len(combined_df['item_id'].unique()))

943
1682


In [11]:
user_movie_matrix = combined_df.pivot_table(index='user_id', columns='item_id', values='rating')
user_movie_matrix = user_movie_matrix.fillna(0)
user_movie_matrix

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=5)
user_movie_clusters = kmeans.fit_predict(user_movie_matrix)

print(user_movie_clusters)
print(user_movie_clusters.shape) #each user gets a cluster assignment

[5 1 1 1 5 6 6 3 1 6 9 9 2 9 1 7 1 2 1 1 1 3 5 9 9 1 1 9 1 1 1 1 1 1 1 1 3
 9 1 1 9 5 5 5 1 1 1 9 7 1 1 1 1 1 1 5 1 7 2 6 1 7 1 5 9 1 1 1 1 5 9 7 9 1
 1 9 9 1 1 1 1 9 5 1 6 1 5 1 1 2 9 4 1 4 5 9 9 1 7 1 1 3 1 1 1 9 1 1 5 3 1
 1 1 9 7 1 7 9 7 1 9 9 9 1 5 1 1 0 1 4 1 1 1 1 1 1 1 9 1 1 1 1 1 7 7 1 1 9
 1 1 0 9 1 9 1 9 1 3 1 7 9 1 1 1 1 1 9 1 1 1 1 1 1 7 9 1 7 4 1 9 1 1 1 0 1
 1 9 9 6 1 1 1 1 6 1 1 3 5 1 5 2 1 1 1 1 1 7 9 1 0 1 1 7 7 9 7 3 9 1 1 7 4
 1 1 9 9 1 1 1 5 1 9 9 6 9 9 9 1 6 1 1 1 9 5 1 5 1 9 7 7 1 1 9 5 1 3 1 1 1
 1 1 9 6 7 1 1 5 5 2 7 6 9 1 1 9 4 1 1 5 5 1 1 9 1 1 5 1 9 1 5 4 7 2 1 5 7
 7 9 2 1 5 1 4 1 2 1 5 2 1 1 6 6 6 5 9 9 1 0 1 3 9 9 7 1 9 6 2 6 1 5 1 3 1
 2 1 1 1 9 2 9 1 7 2 7 7 3 5 1 1 9 1 9 1 2 1 1 1 1 1 9 9 1 5 1 1 1 1 1 1 9
 9 1 5 5 1 1 1 4 0 9 9 1 9 1 2 1 2 1 0 1 9 9 8 5 9 1 9 0 5 1 9 1 1 1 0 6 5
 1 6 1 9 9 1 1 1 4 4 1 1 1 9 1 1 1 3 9 1 1 4 1 1 1 1 1 4 7 0 1 1 1 1 3 1 1
 1 1 7 1 1 4 1 6 7 6 7 7 4 7 1 1 1 1 1 1 9 3 1 7 9 1 1 5 1 2 1 9 1 7 5 9 9
 1 1 5 1 1 5 9 1 1 1 9 7 

## STEP 3: BUILDING & TRAINING THE LINUCB MODEL

In [13]:
# Implementation assisted by ChatGPT

import numpy as np

class LinUCB:
  def __init__(self, n_arms, context_dim, alpha):
    self.n_arms = n_arms
    self.context_dim = context_dim
    self.alpha = alpha
    self.A = [np.identity(context_dim) for arm in range(n_arms)]
    self.b = [np.zeros(context_dim) for arm in range(n_arms)]

  def score(self, arm, x):
        A_inv = np.linalg.inv(self.A[arm])
        theta = A_inv @ self.b[arm]
        score = np.transpose(theta) @ x + self.alpha * np.sqrt(np.transpose(x) @ A_inv @ x)
        return score

  def select_arm(self, x):
    p_vals = []
    for i in range(self.n_arms):
        p = self.score(i, x)
        p_vals.append(p)
    return np.argmax(p_vals)

  def update(self, arm_idx, x, r):
    self.A[arm_idx] += np.outer(x, x)
    self.b[arm_idx] += r * x

In [14]:
n_training = int(combined_df.shape[0]*0.99)

training_df = combined_df.iloc[:n_training]
testing_df = combined_df.iloc[n_training:]

In [15]:
num_clusters = 10
context_dim = user_movie_feature.shape[1]
linucb = LinUCB(num_clusters, context_dim, 0.5)

for idx, row in training_df.iterrows():
  user_id = row['user_id']
  context = user_movie_feature.iloc[idx]
  reward = row['rating']
  std_reward = reward/5

  user_cluster = user_movie_clusters[user_id-1]

  linucb.update(user_cluster, context, std_reward)

In [16]:
# Implementation assisted by ChatGPT

def ndcg(relevance_scores):

    dcg = np.sum((2**relevance_scores - 1) / np.log2(np.arange(2, relevance_scores.size + 2)))

    ideal_relevance = np.array(sorted(relevance_scores, reverse=True))
    idcg = np.sum((2**ideal_relevance - 1) / np.log2(np.arange(2, ideal_relevance.size + 2)))

    ndcg = dcg / idcg

    return ndcg

## Testing

Because we want to simulate a cold start problem, we assume that the testing data for new users only contain the user's age, occupation, and genre.

In [17]:
test_rewards = []
cumulative_test_reward = 0
test_arm_selection = []

for idx, row in testing_df.iterrows():
  print(row)
  context = user_feature.iloc[idx]
  print(context)
  reward = row['rating']
  std_reward = reward/5

  #chosen_arm = linucb.select_arm(context)
  #test_arm_selection.append(chosen_arm)

  #cumulative_test_reward += std_reward
  #test_rewards.append(std_reward)

  break

#ndcg(np.array(test_rewards))

user_id            774
item_id            391
rating               1
age                 30
gender               M
occupation     student
Action               1
Adventure            0
Animation            0
Children             0
Comedy               1
Crime                0
Documentary          0
Drama                0
Fantasy              0
Film-Noir            0
Horror               0
Musical              0
Mystery              0
Romance              0
Sci-Fi               0
Thriller             0
War                  0
Western              0
Name: 99000, dtype: object
age                         0.348485
gender_F                    0.000000
gender_M                    1.000000
occupation_administrator    0.000000
occupation_artist           0.000000
occupation_doctor           0.000000
occupation_educator         0.000000
occupation_engineer         0.000000
occupation_entertainment    0.000000
occupation_executive        0.000000
occupation_healthcare       0.000000
occupation_hom

In [18]:
def dcg(relevances):
    return sum([(2**rel - 1) / np.log2(idx + 2) for idx, rel in enumerate(relevances)])

def ndcg_at_k(true_relevance, predicted_scores, k=10):
    idx_sorted = np.argsort(predicted_scores)[::-1][:k]
    relevances = [true_relevance[i] for i in idx_sorted]
    ideal_relevances = sorted(true_relevance, reverse=True)[:k]
    return dcg(relevances) / dcg(ideal_relevances) if dcg(ideal_relevances) > 0 else 0

In [19]:
ndcg_scores = []

for idx, row in testing_df.iterrows():
    user_id = row['user_id']
    true_movie = row['item_id']
    true_rating = row['rating']

    # Only user features
    user_vec = user_feature.iloc[idx]

    predicted_score = []
    true_relevance = []

    for idx, movie_row in movie.iterrows():
      movie_vec = movie_row.drop('item_id')
      context = np.concatenate([user_vec, movie_vec])
      arm = linucb.select_arm(context)

      score = linucb.score(arm,context)
      predicted_score.append(score)

      if movie_row['item_id'] == true_movie:
          true_relevance.append(true_rating)
      else:
          true_relevance.append(0)

    ndcg = ndcg_at_k(true_relevance, predicted_score, k=10)
    ndcg_scores.append(ndcg)

print(f"Average NDCG@10: {np.mean(ndcg_scores):.4f}")

KeyboardInterrupt: 